Cross lingual machine translation using Seq2Seq model(LSTM)+Attention:

Pipeline:

*   **Encoder**:Reads the source sentence word by word and outputs hidden state for each word.
*   **Attention**:Lets the decoder focus on relevant encoder outputs(hidden states) for each target word.
*   **Decoder**: Generates the target sentence word by word using attention+ words



1.   **Encoder**: Reads token embeddings and generates hidden state(ht) and cell state(ct). Each ht is passed to attention and decoder.
2.   **Attention Mechanism(Bahdanau)**:The decoder needs to focus on relevant content. hence attention mech gives an attention score for every word. The attention score is a function of ht and decoder state at previous step.Then an attention weight is generated for every word and finally a context vector which is a sum of all (attention wt*ht).
3. **Decoder**:At each step, the  decoder computes decoder hidden state St and throws a probablity of next word to occur.








this code is trained to convert english in any of the 15 languages in opus_books dataset:


1.   ca

1.   de

1.   el

1.   en

1.   eo


1.   es

1.   fi

1.   fr


1.   hu

1.   it

1.   nl

1.   no
2.   pl


2.   pt


2.   sv









In [ ]:
!pip install datasets

In [1]:
from datasets import load_dataset

ModuleNotFoundError: No module named 'datasets'

In [ ]:
target_lan="fr"
combo="en"+"-"+target_lan
combo

'en-fr'

In [ ]:
dataset = load_dataset("opus_books", combo)
train_data = dataset["train"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

en-fr/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [ ]:
src_sentences = [x['translation']['en'] for x in train_data]
tgt_sentences = [x['translation'][target_lan] for x in train_data]

Preprocessing and vocabulary

In [ ]:
!pip install torchtext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.9 MB/s eta 0:00:00


In [ ]:
!pip show torch torchtext


Name: torch
Version: 2.8.0+cu126
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.12/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-cufile-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, setuptools, sympy, triton, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchdata, torchtext, torchvision
---
Name: torchtext
Version: 0.18.0
Summary: Text utilities, models, transforms, and datasets for PyTorch.
Home-page: https://github.com/pytorch/text
Author: PyTorch Text Team
Author-email: packages@pytorch.org
License:

In [ ]:
import torch
from collections import Counter

In [ ]:
import re

def tokenizer(text):
    return re.findall(r"\b\w+\b", text.lower())

# usage
tokens = tokenizer("I went to the market.")
print(tokens)

['i', 'went', 'to', 'the', 'market']


In [ ]:
def build_vocab(sentences, tokenizer, max_size=10000):
    counter = Counter()
    for sent in sentences:
        counter.update((sent.lower()))
    vocab = {word:i+4 for i, (word, freq) in enumerate(counter.most_common(max_size))}
    vocab["<pad>"] = 0
    vocab["<unk>"] = 1
    vocab["<sos>"] = 2
    vocab["<eos>"] = 3
    return vocab

src_vocab = build_vocab(src_sentences, tokenizer)
tgt_vocab = build_vocab(tgt_sentences, tokenizer)

**DATASET CLASS**

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
from torch.utils.data import Dataset, DataLoader

class TranslationDataset(Dataset):
    def __init__(self, src_sentences, tgt_sentences, src_vocab, tgt_vocab, tokenizer):
        self.src = src_sentences
        self.tgt = tgt_sentences
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        src_tokens = [self.src_vocab.get(tok, self.src_vocab["<unk>"]) for tok in self.tokenizer(self.src[idx].lower())]
        tgt_tokens = [self.tgt_vocab["<sos>"]] + \
                     [self.tgt_vocab.get(tok, self.tgt_vocab["<unk>"]) for tok in self.tokenizer(self.tgt[idx].lower())] + \
                     [self.tgt_vocab["<eos>"]]
        return torch.tensor(src_tokens), torch.tensor(tgt_tokens)

# DataLoader with padding
def collate_fn(batch):
    src_batch = torch.nn.utils.rnn.pad_sequence([x[0] for x in batch], batch_first=True, padding_value=src_vocab["<pad>"])
    tgt_batch = torch.nn.utils.rnn.pad_sequence([x[1] for x in batch], batch_first=True, padding_value=tgt_vocab["<pad>"])
    return src_batch, tgt_batch

train_dataset = TranslationDataset(src_sentences, tgt_sentences, src_vocab, tgt_vocab, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)


**Seq2Seq Model with Attention**

**ENCODER**

In [ ]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, n_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hidden_dim, n_layers, batch_first=True)

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, (hidden, cell) = self.rnn(embedded)
        return outputs, hidden, cell


**ATTENTION**

In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim*2, hidden_dim)
        self.v = nn.Parameter(torch.rand(hidden_dim))

    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.size(1)
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        energy = energy @ self.v
        return nn.functional.softmax(energy, dim=1)


**DECODER**

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, attention, n_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim + hidden_dim, hidden_dim, n_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim*2, output_dim)
        self.attention = attention

    def forward(self, input, hidden, cell, encoder_outputs):
        input = input.unsqueeze(1)
        embedded = self.embedding(input)
        attn_weights = self.attention(hidden[-1], encoder_outputs)
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs)
        rnn_input = torch.cat((embedded, context), dim=2)
        outputs, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))
        prediction = self.fc_out(torch.cat((outputs.squeeze(1), context.squeeze(1)), dim=1))
        return prediction, hidden, cell


**TRAINING LOOP**

In [ ]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(len(src_vocab), 256, 512).to(device)
attention = Attention(512)
decoder = Decoder(len(tgt_vocab), 256, 512, attention).to(device)

optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()))
criterion = nn.CrossEntropyLoss(ignore_index=tgt_vocab["<pad>"])

num_epochs = 5

for epoch in range(num_epochs):
    encoder.train()
    decoder.train()
    epoch_loss = 0
    for src_batch, tgt_batch in train_loader:
        src_batch, tgt_batch = src_batch.to(device), tgt_batch.to(device)
        optimizer.zero_grad()
        encoder_outputs, hidden, cell = encoder(src_batch)
        input_token = tgt_batch[:,0]
        loss = 0
        for t in range(1, tgt_batch.size(1)):
            output, hidden, cell = decoder(input_token, hidden, cell, encoder_outputs)
            loss += criterion(output, tgt_batch[:,t])
            input_token = tgt_batch[:,t]  # teacher forcing
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")


In [ ]:
def translate(sentence, encoder, decoder, src_vocab, tgt_vocab, tokenizer, max_len=50):
    encoder.eval()
    decoder.eval()
    src_tokens = torch.tensor([src_vocab.get(tok, src_vocab["<unk>"]) for tok in tokenizer(sentence.lower())]).unsqueeze(0).to(device)
    encoder_outputs, hidden, cell = encoder(src_tokens)
    input_token = torch.tensor([tgt_vocab["<sos>"]]).to(device)
    translated = []
    inv_tgt_vocab = {v:k for k,v in tgt_vocab.items()}
    for _ in range(max_len):
        output, hidden, cell = decoder(input_token, hidden, cell, encoder_outputs)
        pred_token = output.argmax(1).item()
        if pred_token == tgt_vocab["<eos>"]:
            break
        translated.append(inv_tgt_vocab[pred_token])
        input_token = torch.tensor([pred_token]).to(device)
    return " ".join(translated)

# Example
print(translate("Hello, how are you?", encoder, decoder, src_vocab, tgt_vocab, tokenizer))
